# Sequence to Sequence RNN with Attention

### Loading Libraries

In [1]:
%cd ../..

/Users/joaquinromero/Desktop


In [ ]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# OS 
import os
import shutil
import joblib

# Data Visualization
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

# Path
from pathlib import Path
from tqdm.autonotebook import tqdm

# IPython & Itertools
from itertools import cycle
from IPython.display import display, HTML

In [ ]:
# Custom Libraries
from src.utils import plotting_utils
from src.utils import ts_utils_updated
from src.forecasting.ml_forecasting import calculate_metrics

from src.forecasting.ml_forecasting import (
    MissingValueConfig,
    calculate_metrics,
)

In [ ]:
# %load_ext autoreload

# %autoreload 2

In [ ]:
tqdm.pandas()

np.random.seed(42)

pio.templates.default = "plotly_white"

In [ ]:
os.makedirs("imgs/chapter_13", exist_ok=True)

preprocessed = Path.home() / "Desktop" / "data" / "london_smart_meters" / "preprocessed"

output = Path.home() / "Desktop" / "data" / "london_smart_meters" / "output"

#### Utility Functions